**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("adult_income.zip"), directory="data/adult_income")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Rozhodovacie stromy pre klasifikáciu: Adult Income Dataset

Po tom ako sme ilustrovali fungovanie klasifikátorov na báze rozhodovacích stromov na dátovej množine Titanic, prejdeme si ešte jeden príklad: tento raz na dátovej množine [Adult Income Dataset](https://archive.ics.uci.edu/ml/datasets/adult). Keďže ide opäť o dátovú množinu, s ktorou sme už pracovali, kód potrebný na načítanie a predspracovanie dát uvádzame v nasledujúcej bunke a je skrytý v záujme stručnosti.



In [ ]:
#@title -- Loading and Preprocessing the Data: X_train, Y_train, X_test, Y_test -- { display-mode: "form" }
df_train = pd.read_csv("data/adult_income/adult.data",
                       header=None)
df_test = pd.read_csv("data/adult_income/adult.test",
                      header=None, skiprows=1)
df_test[14] = df_test[14].apply(lambda x: x[:-1])
    
categorical_inputs = [1, 3, 5, 6, 7, 8, 9, 13]
numeric_inputs = [0, 2, 4, 10, 11, 12]

output = 14

input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

output_enc = OrdinalEncoder()
    
X_train = input_preproc.fit_transform(df_train)
Y_train = output_enc.fit_transform(df_train[[output]]).reshape(-1)

X_test = input_preproc.transform(df_test)
Y_test = output_enc.transform(df_test[[output]]).reshape(-1)

---
### Úloha 1: Applikácia rozhodovacieho stromu a ladenie jeho hyperparametrov

**Dátová množina je uložená v poliach `X_train`, `Y_train`, `X_test`, `Y_test`. Aplikujte na ňu rozhodovací strom a jeho hyperparametre nalaďte pomocou krížovej validácie na tréningovej množine. Potom ako sa hyperparametre vyladia, finálny model natrénujte na celej tréningovej dátovej množine.** 

POZN. 1: Možno budete mať tendenciu jednoducho použiť hyperparametre z predchádzajúceho príkladu. Nie je vylúčené, že budú fungovať. Vo všeobecnosti však treba povedať, že optimálne hyperparametre závisia aj od dátovej množiny, nielen od použitého modelu.

POZN. 2: Ak sa rozhodnete vizualizovať rozhodovací strom, postupujte opatrne. Ak sa neaplikuje dostatočná miera prerezávania, strom môže byť priveľký na to, aby sa dal pohodlne vizualizovať a jeho vykreslenie môže trvať neprakticky dlho.

---


In [ ]:


# ----



### Testovanie

Potom ako sme vyladili hyperparametre a natrénovali model, môžeme overiť zovšeobecnenie na testovacích dátach.



In [ ]:
y_test = model.predict(X_test)

cm = pd.crosstab(Y_test, y_test,
                 rownames=['actual'],
                 colnames=['predicted'])
print(cm, "\n")

acc = accuracy_score(Y_test, y_test)
print("Accuracy = {}".format(acc))